# Implementasi model ARIMA dalam anggaran belanja

# 1. Business Understanding
#### Tujuan: Meramalkan anggaran (belanja) menggunakan model ARIMA berdasarkan data historis.

# 2. Data Understanding
#### Dataset: anggaran.csv dengan atribut
#### Pendapatan: Data pendapatan (tidak digunakan dalam model ini).
#### Belanja: Anggaran yang akan diprediksi.
#### Date: Tanggal yang digunakan sebagai indeks waktu.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
# Load dataset
data = pd.read_csv('anggaran.csv')

In [ ]:
# Data Understanding
# Display first few rows of the training data
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
# Convert 'Date' column to datetime format and set it as index
# Mengubah kolom 'Date' menjadi format datetime
data['Date'] = pd.to_datetime(data['Date'])

# Mengatur kolom 'Date' sebagai index dataframe
# inplace=True berarti perubahan dilakukan langsung pada dataframe asli
data.set_index('Date', inplace=True)

# 3. Data Preparation
#### Pastikan data dalam format datetime dan lakukan pengecekan stasioneritas. Jika data tidak stasioner, lakukan differencing.

In [ ]:
# Melakukan uji Augmented Dickey-Fuller (ADF) pada data 'Belanja'
result = adfuller(data['Belanja'])

# Mengambil nilai statistik ADF dan p-value dari hasil uji
adf_stat, p_value = result[0], result[1]

# Mencetak hasil uji
print(f"ADF Statistic: {adf_stat}")
print(f"P-Value: {p_value}")

In [ ]:
# Jika p-value > 0,05, maka data tidak stasioner dan mungkin perlu dilakukan differencing
if p_value > 0.05:
    print("Data is not stationary. Differencing is applied.")
    # Melakukan differencing pada data 'Belanja' dan menyimpan hasilnya dalam kolom baru 'Belanja_diff'
    # dropna() digunakan untuk menghapus nilai NaN yang dihasilkan oleh differencing
    data['Belanja_diff'] = data['Belanja'].diff().dropna()

# 4. Modelling
#### Membuat model ARIMA dengan parameter (p, d, q):
#### p: Order untuk Autoregressive (AR).
#### d: Order untuk differencing.
#### q: Order untuk Moving Average (MA).
#### Melakukan prediksi untuk beberapa langkah ke depan (forecast horizon).

#### Terapkan model ARIMA dengan parameter (p=1, d=0, q=1) untuk meramalkan anggaran (belanja).

In [ ]:
# Menerapkan model ARIMA dengan parameter awal (p=1, d=0, q=1)
# p: jumlah autoregressive terms
# d: jumlah differencing yang diperlukan untuk membuat data stasioner
# q: jumlah moving average terms
model = ARIMA(data['Belanja'], order=(1, 0, 1))  # p=1, d=0, q=1

# Melakukan fitting model ARIMA pada data 'Belanja'
fitted_model = model.fit()

In [ ]:
# Membuat prediksi untuk 10 langkah ke depan
forecast_horizon = 10

# Menggunakan metode forecast() dari model ARIMA yang telah di-fitting
# steps: jumlah langkah ke depan yang ingin diprediksi
forecast = fitted_model.forecast(steps=forecast_horizon)

In [ ]:
# Membuat indeks untuk prediksi
# start: tanggal awal prediksi, yaitu tanggal terakhir data asli
# periods: jumlah periode yang ingin diprediksi, yaitu forecast_horizon + 1
# freq: frekuensi prediksi, yaitu bulanan ('M')
# [1:]: menghilangkan tanggal awal yang sama dengan tanggal terakhir data asli
forecast_index = pd.date_range(start=data.index[-1], periods=forecast_horizon + 1, freq='M')[1:]

# 5. Evaluation
#### Evaluasi hasil dengan melihat prediksi untuk periode berikutnya dan bandingkan dengan data historis (jika ada).


In [ ]:
# Menampilkan nilai prediksi
# Mengubah hasil prediksi menjadi dataframe dengan nama 'Forecast'
forecast_results = forecast.to_frame(name='Forecast')

# Mengatur indeks dataframe dengan indeks prediksi yang telah dibuat
forecast_results.index = forecast_index

# Menampilkan dataframe prediksi
forecast_results

# 6. Deployment
#### Visualisasikan hasil prediksi.

In [ ]:
# Visualisasi data aktual dan prediksi
# Membuat plot dengan ukuran 12x6 inci
plt.figure(figsize=(12, 6))

# Plot data aktual dengan label 'Historical Data'
plt.plot(data['Belanja'], label='Historical Data')

# Plot prediksi dengan label 'Forecast', linestyle '--' (garis putus-putus), dan warna merah
plt.plot(forecast_index, forecast, label='Forecast', linestyle='--', color='red')

# Menambahkan judul plot
plt.title(f'ARIMA Model - {forecast_horizon}-Step Forecast')

# Menambahkan label pada sumbu x
plt.xlabel('Date')

# Menambahkan label pada sumbu y
plt.ylabel('Belanja')

# Menambahkan legenda
plt.legend()

# Menambahkan grid pada plot
plt.grid(True)

# Menampilkan plot
plt.show()